# Training Material: ALOS-2 Layer Stacking

## Module 1: Layer Stacking (specifying individual file paths)

To create a layer stacking using ALOS-2 data, you need to align multiple images (from different acquisition dates or polarizations) into a single multi-band GeoTIFF. Here's a step-by-step guide.

## I. Prerequisites

* **Basic Python for Data Processing:**
    * Data: ALOS-2 images (e.g., GeoTIFFs or other formats) calibrated and clipped to the Area of Interest (AOI).
    * Software: Use Python with libraries like rasterio, numpy, and gdal.

In [1]:
pip install rasterio matplotlib numpy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## II. Load ALOS-2 Images

In [2]:
import rasterio
import numpy as np

# Paths to ALOS-2 images (e.g., GeoTIFFs)
image_paths = [
    "/home/jovyan/shared/Sirinya/gen_ai/preprocessed/HH-191127_filtered.tif",  # Date 1
    "/home/jovyan/shared/Sirinya/gen_ai/preprocessed/HH-200610_filtered.tif",  # Date 2
    "/home/jovyan/shared/Sirinya/gen_ai/preprocessed/HH-210609_filtered.tif",  # Date 3
]

# Open the images and ensure alignment
images = [rasterio.open(img) for img in image_paths]


## (Optional) Check Alignment

Ensure all images have the same dimensions, resolution, and CRS (coordinate reference system).

In [4]:
for img in images:
    print(f"CRS: {img.crs}, Shape: {img.shape}, Resolution: {img.res}")

# Check extents
bounds = [img.bounds for img in images]
if len(set(bounds)) > 1:
    print("Warning: Images are not aligned! Reprocess to align them.")


CRS: PROJCS["Geo-coded",GEOGCS["ITRF97 (geocentric)",DATUM["International_Terrestrial_Reference_Frame_1997",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6655"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4338"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",135],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]], Shape: (1604, 1556), Resolution: (2.5, 2.5)
CRS: PROJCS["Geo-coded",GEOGCS["ITRF97 (geocentric)",DATUM["International_Terrestrial_Reference_Frame_1997",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6655"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4338"]],PROJECTION["Transverse_Mercator"],PARAMETER["l

## III. Stack Layers

In [3]:
# Read image data and stack layers
stacked_data = np.stack([img.read(1) for img in images], axis=0)

# Print the shape of stacked data (should be [num_layers, height, width])
print(f"Stacked Data Shape: {stacked_data.shape}")


Stacked Data Shape: (3, 1604, 1556)


## IV. Save Stacked Layers to a New GeoTIFF

Write the stacked array to a new GeoTIFF file.

In [5]:
output_path = "/home/jovyan/shared/Sirinya/gen_ai/stacked_image.tif"

# Use the metadata from one of the input images
meta = images[0].meta.copy()
meta.update({
    "count": len(images),  # Number of layers
    "dtype": stacked_data.dtype,
})

# Write stacked layers
with rasterio.open(output_path, "w", **meta) as dest:
    for i in range(stacked_data.shape[0]):
        dest.write(stacked_data[i, :, :], i + 1)  # Write each band
print(f"Stacked image saved at {output_path}")


Stacked image saved at /home/jovyan/shared/Sirinya/gen_ai/stacked_image.tif


## Module 2: Layer Stacking to work with a folder path instead of specifying individual file paths

## I. Load ALOS-2 Images

In [3]:
import os
import rasterio
import numpy as np

# Specify the folder containing ALOS-2 images
folder_path = "/home/jovyan/shared/genAI/3_sirin/preprocessed"

# List all GeoTIFF files in the folder
image_paths = [
    os.path.join(folder_path, file)
    for file in os.listdir(folder_path)
    if file.endswith(".tif")  # Filter only GeoTIFF files
]

# Sort the file paths (optional, ensures consistent stacking order)
image_paths.sort()

# Open the images
images = [rasterio.open(img) for img in image_paths]

# Print file names to verify
print(f"Loaded images: {image_paths}")


Loaded images: ['/home/jovyan/shared/genAI/3_sirin/preprocessed/HH-191127_filtered.tif', '/home/jovyan/shared/genAI/3_sirin/preprocessed/HH-200304_filtered.tif', '/home/jovyan/shared/genAI/3_sirin/preprocessed/HH-200610_filtered.tif', '/home/jovyan/shared/genAI/3_sirin/preprocessed/HH-210303_filtered.tif', '/home/jovyan/shared/genAI/3_sirin/preprocessed/HH-210609_filtered.tif', '/home/jovyan/shared/genAI/3_sirin/preprocessed/HH-211124_filtered.tif', '/home/jovyan/shared/genAI/3_sirin/preprocessed/HH-220302_filtered.tif']


## II. Stack Layers

In [4]:
# Stack the layers from multiple images
stacked_data = np.stack([img.read(1) for img in images], axis=0)
print("Image layers stacked successfully.")


Image layers stacked successfully.


## III. Save the stacked layers to a new GeoTIFF

In [6]:
# Define the output path for the stacked GeoTIFF
output_path = "/home/jovyan/shared/genAI/3_sirin/stacked_image_2.tif"

# Copy metadata from the first image and update for the stacked layers
meta = images[0].meta.copy()
meta.update({
    "count": len(images),  # Number of layers
    "dtype": stacked_data.dtype,
})

# Save the stacked layers to a new GeoTIFF
with rasterio.open(output_path, "w", **meta) as dest:
    for i in range(stacked_data.shape[0]):
        dest.write(stacked_data[i, :, :], i + 1)  # Write each band
print(f"Stacked image saved at {output_path}")


Stacked image saved at /home/jovyan/shared/genAI/3_sirin/stacked_image_2.tif
